# Diametrics Package Demo

Diametrics is ...

<iframe width="580" height="364" src="https://www.youtube.com/embed/uUcuigkCrm0" title="Welcome!" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" allowfullscreen></iframe>

In [2]:
import pandas as pd
import numpy as np
#from diametrics import transform
from src.diametrics import transform, metrics, preprocessing, visualizations
#import transformData
import copy

SyntaxError: invalid syntax (53305568.py, line 4)

## 1. Uploading and transforming data

#### 1.1. By directory
The most common use case is transforming a directory of CGM files into one dataframe

In [2]:
dexcom_data = transform.transform_directory(directory='tests/test_data/dexcom/', device='dexcom')
dexcom_data.head()

,time,glc,ID
0,2023-03-08 00:04:00,6.4,dexcom_eur_01
1,2023-03-08 00:09:00,6.5,dexcom_eur_01
2,2023-03-08 00:13:59,6.1,dexcom_eur_01
3,2023-03-08 00:18:59,6.5,dexcom_eur_01
4,2023-03-08 00:23:59,6.3,dexcom_eur_01


#### Individual files

In [3]:
libre1 = transform.open_file('tests/test_data/libre/libre_amer_01.csv')
libre1_transformed = transform.convert_libre(libre1)
libre1_transformed.head()

,time,glc,scan_glc
0,2021-03-20 17:38:00,127,NaN
1,2021-03-20 17:53:00,124,NaN
2,2021-03-20 18:08:00,121,NaN
3,2021-03-20 18:23:00,131,NaN
4,2021-03-20 18:38:00,153,NaN


In [4]:
dxcm2 = transform.open_file('tests/test_data/dexcom/dexcom_eur_02.xlsx')
dxcm2_transformed = transform.convert_dexcom(dxcm2)
dxcm2_transformed.head()

,time,glc
0,2023-03-08 00:00:44,10.4
1,2023-03-08 00:05:44,10.3
2,2023-03-08 00:10:44,10.2
3,2023-03-08 00:15:44,10.1
4,2023-03-08 00:20:44,9.9


## Preprocessing

#### Check df is going to be usable

In [5]:
preprocessing.check_df(dexcom_data)


True

In [6]:
preprocessing.check_df(dxcm2_transformed)


True

#### Replacing the lo/hi cutoff values

In [7]:
dexcom_data = preprocessing.replace_cutoffs(dexcom_data)

In [8]:
libre1_transformed = preprocessing.replace_cutoffs(libre1_transformed, lo_cutoff=2.1, hi_cutoff=27.8)

In [9]:
dxcm2_transformed = preprocessing.replace_cutoffs(dxcm2_transformed, remove=True)

#### Change start and end times

##### Individual with one start time for all

In [10]:
preprocessing.set_time_frame(libre1_transformed, ['2021-03-21', '2021-04-01'])

,time,glc,scan_glc
26,2021-03-21 00:08:00,188,NaN
27,2021-03-21 00:23:00,178,NaN
28,2021-03-21 00:38:00,163,NaN
29,2021-03-21 00:53:00,160,NaN
30,2021-03-21 01:08:00,158,NaN
...,...,...,...
1077,2021-03-31 22:53:00,162,NaN
1078,2021-03-31 23:08:00,156,NaN
1079,2021-03-31 23:23:00,155,NaN
1080,2021-03-31 23:38:00,145,NaN


##### Using a dictionary

In [11]:
d = {'dexcom_eur_01': ['2023-03-13 03:00:00', '2023-03-20 14:30:00'],
     'dexcom_eur_02': ['2023-03-09', '2023-03-18'],
     'dexcom_eur_03': ['2023-03-14', '2023-03-21']}

In [12]:
preprocessing.set_time_frame(dexcom_data, d)

,time,glc,ID
0,2023-03-13 03:04:11,14.7,dexcom_eur_01
1,2023-03-13 03:09:11,14.7,dexcom_eur_01
2,2023-03-13 03:14:11,14.2,dexcom_eur_01
3,2023-03-13 03:19:11,13.8,dexcom_eur_01
4,2023-03-13 03:24:11,13.8,dexcom_eur_01
...,...,...,...
6562,2023-03-20 23:35:18,7.2,dexcom_eur_03
6563,2023-03-20 23:40:18,7.3,dexcom_eur_03
6564,2023-03-20 23:45:18,7.3,dexcom_eur_03
6565,2023-03-20 23:50:18,7.3,dexcom_eur_03


#### Interpolate missing data

##### Create synthetic missing data

In [13]:
# Create a chunk of data from the transformed DataFrame
dxcm2_chunk = copy.copy(dxcm2_transformed.head(30))
# Set certain rows in the 'glc' column to NaN to simulate missing data
dxcm2_chunk.loc[[4,5,14,15,16,17,18,19,20,21,26,27],'glc'] = np.nan
# Display the first 10 rows of the chunk DataFrame
dxcm2_chunk.head(10)

,time,glc
0,2023-03-08 00:00:44,10.4
1,2023-03-08 00:05:44,10.3
2,2023-03-08 00:10:44,10.2
3,2023-03-08 00:15:44,10.1
4,2023-03-08 00:20:44,NaN
5,2023-03-08 00:25:44,NaN
6,2023-03-08 00:30:44,9.9
7,2023-03-08 00:35:44,9.9
8,2023-03-08 00:40:44,9.5
9,2023-03-08 00:45:44,9.5


In [14]:
# Fill missing data using the preprocessing function
preprocessing.fill_missing_data(dxcm2_chunk, interval=5, method='pchip', limit=30).head(10)

,glc
0,10.4
1,10.3
2,10.2
3,10.1
4,10.0
5,9.9
6,9.9
7,9.9
8,9.5
9,9.5


In [15]:
libre1_chunk = copy.copy(libre1_transformed.head(30))
libre1_chunk.loc[[4,5,14,15,16,17,18,19,20,21,26,27],'glc'] = np.nan
libre1_chunk.head(10)

,time,glc,scan_glc
0,2021-03-20 17:38:00,127.0,NaN
1,2021-03-20 17:53:00,124.0,NaN
2,2021-03-20 18:08:00,121.0,NaN
3,2021-03-20 18:23:00,131.0,NaN
4,2021-03-20 18:38:00,NaN,NaN
5,2021-03-20 18:53:00,NaN,NaN
6,2021-03-20 19:08:00,166.0,NaN
7,2021-03-20 19:23:00,165.0,NaN
8,2021-03-20 19:38:00,162.0,NaN
9,2021-03-20 19:53:00,154.0,NaN


In [16]:
preprocessing.fill_missing_data(libre1_chunk, interval=15, method='linear', limit=45).head(10)

,glc,scan_glc
0,127.0,NaN
1,124.0,NaN
2,121.0,NaN
3,131.0,NaN
4,142.7,NaN
5,154.3,NaN
6,166.0,NaN
7,165.0,NaN
8,162.0,NaN
9,154.0,NaN


#### Change units

In [17]:
preprocessing.change_units(libre1_transformed)

,time,glc,scan_glc
0,2021-03-20 17:38:00,7.1,NaN
1,2021-03-20 17:53:00,6.9,NaN
2,2021-03-20 18:08:00,6.7,NaN
3,2021-03-20 18:23:00,7.3,NaN
4,2021-03-20 18:38:00,8.5,NaN
...,...,...,...
1334,2021-04-03 15:08:00,7.0,NaN
1335,2021-04-03 15:23:00,6.6,NaN
1336,2021-04-03 15:38:00,6.1,NaN
1337,2021-04-03 15:53:00,6.0,NaN


In [18]:
preprocessing.change_units(dxcm2_transformed)

,time,glc
0,2023-03-08 00:00:44,187.0
1,2023-03-08 00:05:44,185.0
2,2023-03-08 00:10:44,183.0
3,2023-03-08 00:15:44,181.0
4,2023-03-08 00:20:44,178.0
...,...,...
3890,2023-03-21 15:10:57,117.0
3891,2023-03-21 15:15:57,122.0
3892,2023-03-21 15:20:57,124.0
3893,2023-03-21 15:25:57,129.0


## Visualisations

### Individual figs

In [19]:
visualizations.glucose_trace(dxcm2_transformed)

In [20]:
visualizations.tir_pie(libre1_transformed)

In [21]:
visualizations.agp(dxcm2_transformed)

In [22]:
visualizations.boxplot(dexcom_data)

## Metrics

### Data sufficiency

In [23]:
metrics.data_sufficiency(dxcm2_transformed)

{'Start DateTime': '2023-03-08 00:01:00',
 'End DateTime': '2023-03-21 15:31:00',
 'Data Sufficiency (%)': 99.1}

### Average glucose

In [24]:
metrics.average_glc(libre1_transformed)

126.06721433905899

### Glycemic variability

In [25]:
metrics.glycemic_variability(dxcm2_transformed)

{'SD': 3.1480384322503565, 'CV (%)': 40.523262102884964}

### Time in range

In [26]:
metrics.time_in_range(libre1_transformed)

{'TIR normal': 90.44,
 'TIR normal 1': 67.66,
 'TIR normal 2': 22.78,
 'TIR level 1 hypoglycemia': 0.3,
 'TIR level 2 hypoglycemia': 0.0,
 'TIR level 1 hyperglycemia': 9.26,
 'TIR level 2 hyperglycemia': 0.0}

### Area under the curve

In [27]:
metrics.auc(dxcm2_transformed)

{'avg_hourly_auc': 7.088226129799524,
 'auc_daily_breakdown': date
 2023-03-08    7.246564
 2023-03-09    6.893435
 2023-03-10    6.795095
 2023-03-11    6.930716
 2023-03-12    7.760576
 2023-03-13    7.869531
 2023-03-14    3.861667
 2023-03-15    6.641391
 2023-03-16    7.710509
 2023-03-17    7.150480
 2023-03-18    7.972437
 2023-03-19    7.761687
 2023-03-20    7.633791
 2023-03-21    6.765155
 Name: auc, dtype: float64,
 'auc_hourly_breakdown':            date  hour       auc
 0    2023-03-08     0  9.033333
 1    2023-03-08     1  7.758250
 2    2023-03-08     2  6.091625
 3    2023-03-08     3  5.314764
 4    2023-03-08     4  4.774972
 ..          ...   ...       ...
 322  2023-03-21    11  5.268347
 323  2023-03-21    12  3.593500
 324  2023-03-21    13  3.971778
 325  2023-03-21    14  5.391375
 326  2023-03-21    15  3.389639
 
 [327 rows x 3 columns]}

### eA1c

In [28]:
metrics.ea1c(libre1_transformed)

6.019763565820871

### Mean amplitude of glycemic excursions (MAGE)

In [29]:
metrics.mage(dxcm2_transformed)

{'MAGE': 8.284210526315789}

### Hypoglycemic and hyperglycemic episodes

In [30]:
metrics.glycemic_episodes(libre1_transformed)

{'Total number hypoglycemic events': 1,
 'Number LV1 hypoglycemic events': 1,
 'Number LV2 hypoglycemic events': 0,
 'Number prolonged hypoglycemic events': 0,
 'Avg. length of hypoglycemic events': '0 days 01:00:00',
 'Total time spent in hypoglycemic events': '0 days 01:00:00',
 'Total number hyperglycemic events': 16,
 'Number LV1 hyperglycemic events': 16,
 'Number LV2 hyperglycemic events': 0,
 'Number prolonged hyperglycemic events': 0,
 'Avg. length of hyperglycemic events': '0 days 01:56:15',
 'Total time spent in hyperglycemic events': '1 days 07:00:00'}

### LBGI and HBGI

In [31]:
metrics.bgi(dxcm2_transformed)

{'LBGI': 2.3898403848253946, 'HBGI': 4.878463240581278}

### All standard metrics

In [32]:
metrics.standard_metrics(libre1_transformed)

,Start DateTime,End DateTime,Data Sufficiency (%),Days,Average glucose,eA1c (%),AUC,SD,CV (%),LBGI,...,Number LV2 hypoglycemic events,Number prolonged hypoglycemic events,Avg. length of hypoglycemic events,Total time spent in hypoglycemic events,Total number hyperglycemic events,Number LV1 hyperglycemic events,Number LV2 hyperglycemic events,Number prolonged hyperglycemic events,Avg. length of hyperglycemic events,Total time spent in hyperglycemic events
0,2021-03-20 17:38:00,2021-04-03 16:08:00,100,13 days 22:30:00,126.067214,6.019764,94.382836,36.505584,28.957239,0.675633,...,0,0,0 days 01:00:00,0 days 01:00:00,16,16,0,0,0 days 01:56:15,1 days 07:00:00


In [34]:
dexcom_data.groupby('ID').apply(lambda group: metrics.standard_metrics(group))

,,Start DateTime,End DateTime,Data Sufficiency (%),Days,Average glucose,eA1c (%),AUC,SD,CV (%),LBGI,...,Number LV2 hypoglycemic events,Number prolonged hypoglycemic events,Avg. length of hypoglycemic events,Total time spent in hypoglycemic events,Total number hyperglycemic events,Number LV1 hyperglycemic events,Number LV2 hyperglycemic events,Number prolonged hyperglycemic events,Avg. length of hyperglycemic events,Total time spent in hyperglycemic events
ID,,,,,,,,,,,,,,,,,,,,,,
dexcom_eur_01,0,2023-03-08 00:04:00,2023-03-21 15:29:00,97.3,13 days 15:25:00,9.151752,7.384750,8.291685,3.121628,34.109626,0.301074,...,0,0,0 days 00:31:16,0 days 02:05:02,33,20,13,5,0 days 03:30:37,4 days 19:50:13
dexcom_eur_02,0,2023-03-08 00:01:00,2023-03-21 15:31:00,97.4,13 days 15:30:00,7.866179,6.576213,7.104193,3.085039,39.219034,1.729170,...,9,0,0 days 01:00:58,1 days 02:25:06,25,19,6,1,0 days 03:20:48,3 days 11:40:04
dexcom_eur_03,0,2023-03-08 00:01:00,2023-03-21 15:40:00,95.9,13 days 15:39:00,8.212228,6.793854,7.369449,3.256385,39.652884,1.397236,...,7,0,0 days 01:02:11,0 days 16:35:03,40,28,12,3,0 days 02:11:45,3 days 15:49:59


### Calculate percentiles

In [33]:
metrics.percentiles(dxcm2_transformed)

{'Min. glucose': 2.1,
 '10th percentile': 4.0,
 '25th percentile': 5.3,
 '50th percentile': 7.3,
 '75th percentile': 10.0,
 '90th percentile': 12.0,
 'Max. glucose': 22.1}